In [ ]:
%matplotlib inline  

In [3]:
import casadi as ca
from casadi import *
from forward_kinematics import *
from nao_functions import *
from optimization import *
from read_data import *
import time
import sys  # quite lo de directorio
import qi
import numpy as np
from nao_skills import *
import matplotlib.pyplot as plt

start = time.time()

reconstructed_data=reconstruct()
left_hand_points=reconstructed_data["lhand"]
right_hand_points=reconstructed_data["rhand"]
left_elbow_points=reconstructed_data["lforearm"]
right_elbow_points=reconstructed_data["rforearm"]
N=len(left_hand_points)

joint_indices = {
    "l_arm": slice(14, 20),
    "r_arm": slice(20, 26),
    "f_base": slice(26,32),
    "l_leg": slice(2,8),
    "r_leg": slice(8,14),
}



Dof = 32 # 32 cuando agregue los 6 de la base lotante
Dof_act = 26  #26 qs actuados
#N = 80
q = MX.sym('q',Dof*N) # variable 'q' de cuero completo
tau = MX.sym('tau',Dof_act*N)  # par para articulaciones actuadas
f_contact = MX.sym('f_contact', 3*8*(N))  # fuerzas de contacto (4 por pie)
S = ca.vertcat(ca.DM_eye(26), ca.DM.zeros(6, 26))  # Dimensión: 32x26


f=0 # costo
dt = 0.05
mu = 0.54 # inventado

total_constraints = Dof_act*(N-1) + 12*(N) #+ 8*(N-1) #+ Dof*N
print("Restricciones de g:",total_constraints)

lbg=[0]*total_constraints
ubg=[0]*total_constraints
g=[0]*total_constraints # rebisar bien , posible error 

dt = 0.05
idx = 0

p_rfijo = [0.03, -0.05 , 0]
p_lfijo = [0.03, 0.05 , 0]
r_rfijo =  np.array ([[1, 0, 0],
                      [0, 0, 1],
                      [0, -1, 0]]) 

r_lfijo =  np.array ([[1, 0, 0],
                      [0, 0, 1],
                      [0, -1, 0]]) 



for i in range(N):
    q_i = q[Dof*i :Dof*(i+1)]  
    q_i_larm =  q_i[joint_indices["l_arm"]]
    q_i_rarm =  q_i[joint_indices["r_arm"]]
    q_i_base = q_i[joint_indices["f_base"]]

    f += ca.sumsqr(ca.vertcat(*fkine_left_arm(q_i_larm)[0:3,3])-ca.vertcat(*left_hand_points[i]))   # Mano izquierda
    f +=  ca.sumsqr(ca.vertcat(*fkine_left_elbow(q_i_larm)[0:3,3])-ca.vertcat(*left_elbow_points[i]))  # Codo izquierdo
    f += ca.sumsqr(ca.vertcat(*fkine_right_arm(q_i_rarm)[0:3,3])-ca.vertcat(*right_hand_points[i]))   # Mano derecha
    f +=  ca.sumsqr(ca.vertcat(*fkine_right_elbow(q_i_rarm)[0:3,3])-ca.vertcat(*right_elbow_points[i]))  # Codo derecho

    #' Restriccion phi(q) = 0
    q_i_lleg = q_i[joint_indices["l_leg"]]
    q_i_rleg = q_i[joint_indices["r_leg"]]

    T_lleg = fkine_left_foot_constant(q_i_lleg,q_i_base,'c')
    p_lleg = T_lleg[0:3,3]
    r_lleg = T_lleg[0:3,0:3]

    T_rleg = fkine_right_foot_constant(q_i_rleg,q_i_base,'c')
    p_rleg = T_rleg[0:3,3]
    r_rleg = T_rleg[0:3,0:3]


    for j in range(3):
        g[idx] = p_rleg[j] - p_rfijo[j]
        lbg[idx] = 0
        ubg[idx] = 0
        idx +=1
        
        g[idx] = p_lleg[j] - p_lfijo[j]
        lbg[idx] = 0
        ubg[idx] = 0
        idx +=1

    R_err_left = r_lfijo.T@r_lleg # rotación relativa
    R_err_right = r_rfijo.T@r_rleg 

    theta_err_left = ca.vertcat(R_err_left[2,1] - R_err_left[1,2], R_err_left[0,2] - R_err_left[2,0],
                                R_err_left[1,0] - R_err_left[0,1]) / 2.0  # Aproximación de log(R) para pequeñas rotaciones


    theta_err_right = ca.vertcat(R_err_right[2,1] - R_err_right[1,2], R_err_right[0,2] - R_err_right[2,0],
                                 R_err_right[1,0] - R_err_right[0,1]) / 2.0  # Aproximación de log(R) para pequeñas rotaciones


    for j in range(3):
        g[idx] = theta_err_left[j]
        lbg[idx] = 0.0
        ubg[idx] = 0.0
        idx += 1

        g[idx] = theta_err_right[j]
        lbg[idx] = 0.0
        ubg[idx] = 0.0
        idx += 1

    if i<N-1: #evaluar
        q_i1 = q[Dof*(i+1):Dof*(i+2)]
        dq_i = (q_i1-q_i)/dt

        for j in range(Dof_act):
            g[idx] = dq_i[j] 
            lbg[idx] = dq_min[j]
            ubg[idx] = dq_max[j]
            idx+=1

        # Restriccion de fricción
        #f_i = f_contact[24*i:24*(i+1)]
        #f += 1e-6* ca.dot(f_i, f_i)
        #for j in range(8):  # 8 puntos de contacto
        #    fx = f_i[3*j + 0]
        #    fy = f_i[3*j + 1]
        #    fz = f_i[3*j + 2]
        
            # Friction cone constraint: mu^2 * fz^2 - fx^2 - fy^2 >= 0
        #    g[idx] = mu**2 * fz**2 - fx**2 - fy**2
        #    lbg[idx] = 0.0
        #    ubg[idx] = ca.inf
        #    idx += 1

        #if i<N-2:
            #q_i2 = q[Dof*(i+2):Dof*(i+3)]
            #ddq_i = (q_i2 - 2*q_i1 + q_i)/ dt**2

            #M_i =   #M(q_i1)
            #C_i =   #C(q_i1,dq_i)
            #G_i =   #G(q_i1)
            #J_i =   #Jacobiano_contactos(q_i1)

            #tau_i = tau[Dof_act*i:Dof_act(i+1)]
            #dyn = M_i @ ddq_i + C_i @ dq_i + G_i - J_i.T @ f_i - S @ tau_i


            #for j in range(Dof):
            #    g[idx] = dyn[j]  # Solo evaluamos dinámica en los actuados
            #    lbg[idx] = 0
            #    ubg[idx] = 0
            #    idx +=1

            # Queda restringir las uerzas

print("IDX:",idx)
g_all=ca.vertcat(*g)

nlp = {}
nlp['x'] = ca.vertcat(q, tau, f_contact)
nlp['f'] = f
nlp['g'] = g_all
opts={"max_iter": 200}
opts = {
    'ipopt': {
        'print_level': 0     # 0 = no iteration output
    }
}

solver = nlpsol('solver','ipopt',nlp)

lbq=ca.repmat(q_min,N)
ubq=ca.repmat(q_max,N)

lbtau =ca.repmat(tau_min,N)
ubtau = ca.repmat(tau_max,N)

lbf = ca.repmat([-ca.inf]*3*8,N)
ubf = ca.repmat([ca.inf ]*3*8,N)


data = np.load("warmstart_brazos.npz")

q_warm = data['q_full']
q0= q_warm.reshape((-1, 1))  # (Dof*N, 1)
tau0 = np.zeros(Dof_act*N)
fc0 = np.zeros(24*N)
x_init = ca.vertcat(ca.DM(q0), ca.DM(tau0), ca.DM(fc0))


sol = solver(x0 = x_init,
            lbx=vertcat(lbq,lbtau,lbf),
            ubx=vertcat(ubq,ubtau,ubf),
            lbg=lbg, ubg=ubg)


q_full= (sol['x'][:Dof * N]).reshape((Dof,N))
fuerzas_resultado = (sol['x'][Dof*N + Dof_act*N:]).reshape((24,N))
end = time.time()   
print("Tiempo de optimización: ", end-start)


#np.savez("warmstart_brazos.npz", q_full=q_full)


# Main program begins

if __name__ == '__main__': 
    q_full = np.array(q_full)
    #plt.figure(figsize=(10, 6))
    for i in range(6):
        plt.plot(q_full[20+i,:].flatten(),  label=f'qderecho{i+1}')

    plt.title("Evolución de las articulaciones del brazo derecho")
    plt.xlabel("Paso de tiempo")
    plt.ylabel("Ángulo articular (rad)")
    plt.legend()
    #plt.ylim(-6,6)
    #plt.xlim(0,N)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


    #plt.figure(figsize=(10, 6))
    for i in range(6):
        plt.plot(q_full[14+i,:].flatten(), label=f'qizq{i+1}')

    plt.title("Evolución de las articulaciones del brazo izquierdo")
    plt.xlabel("Paso de tiempo")
    plt.ylabel("Ángulo articular (rad)")
    plt.legend()
    #plt.ylim(-6,6)
    #plt.xlim(0,N)
    plt.grid(True)
    plt.tight_layout()
    plt.show()



def nao_trajectory(q_full):
    robot_ip = "192.168.10.157"  # Replace with your robot's IP
    robot_port = 9559

    # Create an application instance
    app = qi.Application(["NAOqiApp", f"--qi-url=tcp://{robot_ip}:{robot_port}"])
    app.start()  # Start the application
    session = app.session  # Get the session from the application
    try:
        print("Connected to the robot!")
    except Exception as e:
        print(f"Failed to connect to robot: {e}")
    
    finally:
        # Close the Qi session properly
        print("Closing Qi session...")
        # Initialize the node
        q_larm_a=np.array(q_full[joint_indices["l_arm"],0].full().flatten())
        q_rarm_a=np.array(q_full[joint_indices["r_arm"],0].full().flatten())
        
        joints_larm(session,q_larm_a.tolist())
        joints_rarm(session,q_rarm_a.tolist())
        for i in range(N):
            q_larm_a=np.array(q_full[joint_indices["l_arm"],i].full().flatten());q_rarm_a=np.array(q_full[joint_indices["r_arm"],i].full().flatten())
            joints_larm(session,q_larm_a.tolist())
            joints_rarm(session,q_rarm_a.tolist())

        app.stop()


Restricciones de g: 6510
IDX: 6510


: 

In [ ]:
fuerzas_resultado = (sol['x'][Dof*N + Dof_act*N:]).reshape((24,N))
print(fuerzas_resultado)


[[0, 0, 0, ..., 0, 0, 0], 
 [0, 0, 0, ..., 0, 0, 0], 
 [0, 0, 0, ..., 0, 0, 0], 
 ...,
 [0, 0, 0, ..., 0, 0, 0], 
 [0, 0, 0, ..., 0, 0, 0], 
 [0, 0, 0, ..., 0, 0, 0]]


: 

In [ ]:
r_rfijo =  ca.MX.eye(3)
r_lfijo =  ca.MX.eye(3)
resultado = ca.mtimes(r_rfijo.T,r_rfijo)
print(resultado)

ones(3x3,3nz)
